## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import itertools
from sklearn import preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataseteeg = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/vsirazenzadnji.csv",delimiter=' ', header = None)
firstfive = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/firstfive.csv",delimiter=' ', header = None)
#dataseteye = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/participant_all-1_eye.csv",delimiter=' ',header=None)


In [ ]:
X = dataseteeg.iloc[:, :-1].values
Xfive = firstfive.iloc[:, :-1].values

y = dataseteeg.iloc[:, -1].values
yfive = firstfive.iloc[:, -1].values
#Xeye_train = dataseteye.iloc[:, :-1].values
print(len(X))
print(len(y))

for x in range(0):
    X = np.concatenate((X,Xfive), axis=0)
    y= np.concatenate((y,yfive), axis=0)

print(len(X))
print(len(y))

scaler = preprocessing.StandardScaler ()


#Xeye_train=scaler.fit_transform(Xeye_train)


#X = []

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
print(len(X[1]))
print(len(y))

In [ ]:
index = pd.Index(y)
index.value_counts()

In [ ]:
inputs = tf.keras.Input(shape=(X.shape[1],))

l = tf.keras.layers.Dense(64,activation="relu")(inputs)
l = tf.keras.layers.Dense(64,activation="relu")(l)
l = tf.keras.layers.Dense(64,activation="relu")(l)
l = tf.keras.layers.Dense(64,activation="relu")(l)
#expand_dims = tf.expand_dims(inputs, axis=2)

#gru = tf.keras.layers.GRU(256, return_sequences=True)(expand_dims)

#flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(5, activation='softmax')(l)



model = tf.keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())

In [ ]:
dot_img_file = "/content/drive/MyDrive/Colab Notebooks/eeg model/model.png"
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=64,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Rezultat

In [ ]:
testseteeg = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/participant_zadnji_otheremotions.csv",delimiter=' ',header=None)

X_test= testseteeg.iloc[:,:-1].values
y_test= testseteeg.iloc[:,-1].values

In [ ]:
X_test = scaler.fit_transform(X_test)


In [ ]:
model_acc = model.evaluate(X_test, y_test, verbose =0)[1]
print("Natančnost modela: {:.3f}%".format(model_acc*100))

In [ ]:
testseteeg1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/zadnji.csv",delimiter=' ',header=None)

X_test1= testseteeg1.iloc[:,:-1].values
y_test1= testseteeg1.iloc[:,-1].values

X_test1 = scaler.fit_transform(X_test1)
model_acc = model.evaluate(X_test1, y_test1, verbose =0)[1]
print("Natančnost modela: {:.3f}%".format(model_acc*100))

In [ ]:
y_pred = np.array(list(map(lambda x: np.argmax(x) , model.predict(X_test))))
label_dict = {"Gnus":0, "strah":1, "žalost":2 , "nevtralno":3, "veselje":4}
cm = confusion_matrix(y_test,y_pred)
clr = classification_report(y_test, y_pred, target_names=label_dict.keys())

In [ ]:
plt.figure(figsize= (8,8))
sns.heatmap(cm, annot= True, fmt="g",cbar =False, cmap = "Blues")
plt.xticks(np.arange(5)+0.5, label_dict.keys())
plt.yticks(np.arange(5)+0.5, label_dict.keys())
plt.xlabel("napovedano")
plt.ylabel("pravilno")
plt.title("tabela napačnih klasifikacij")
plt.show()

print()

In [ ]:
plot_confusion_matrix(clf, X_test, y_test)  

In [ ]:
plt.figure(figsize= (8,8))
sns.heatmap(cm, annot= True, fmt="g",cbar =False, cmap = "Blues")
plt.xticks(np.arange(5)+0.5, label_dict.keys())
plt.yticks(np.arange(5)+0.5, label_dict.keys())
plt.xlabel("napovedano")
plt.ylabel("dejansko")
plt.title("tabela napačnih klasifikacij")
plt.show()

print()